In [103]:
import pandas as pd
import glob
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score

In [124]:
labels = pd.read_csv("..\edos_labelled_aggregated.csv")

data_type = "dev"
labels = labels[labels["split"] == data_type][["rewire_id", "label_sexist"]]

files = glob.glob(fr"..\ensemble\{data_type}*.csv")
dfs = [pd.read_csv(i) for i in files]

labels = dfs[0].merge(labels, on="rewire_id", how="left")[["rewire_id", "label_sexist"]]
y_true = labels.label_sexist.values

len(dfs)

18

# Soft Voting

In [125]:
probabilities = [i[["sexist", "not sexist"]].values for i in dfs]

def soft_voting(probabilities):
    return np.mean(probabilities, axis=0)

# Use soft voting to compute the final probabilities
soft_probabilities = soft_voting(probabilities)
soft_predictions = np.argmax(soft_probabilities, axis=1)
soft_predictions = ["sexist" if i == 0 else "not sexist" for i in soft_predictions]
soft = f1_score(y_true, soft_predictions, average='macro')
print("Soft voting: ", soft)

Soft voting:  0.8492243670654323


# Hard Voting

In [126]:
# Initialize an empty list to store the predicted labels
label = []

# Loop through each row of the first dataframe
for row in range(len(dfs[0])):
    # Count the number of "sexist" and "not sexist" labels across all dataframes
    sexist_count = not_sexist_count = 0
    for df in dfs:
        if df["label_pred"][row] == "sexist":
            sexist_count += 1
        else:
            not_sexist_count += 1
    
    # Choose the label with the higher count
    if sexist_count >= not_sexist_count:
        label.append("sexist")
    else:
        label.append("not sexist")

# Update the label_pred column of the first dataframe with the predicted labels
df = dfs[0]
df["label_pred"] = label

# Compute and print the F1 score
hard = f1_score(y_true, df.label_pred.values, average='macro')
print("hard voting: ", hard)


hard voting:  0.8505323088002501


# highest probability

In [127]:
for i in range(len(dfs)):
    dfs[i]["confidence"] = abs(dfs[i]["sexist"] - dfs[i]["not sexist"])
    dfs[i] = dfs[i][["rewire_id", "label_pred", "confidence"]]
    
labels_position = []
for row in range(len(dfs[0])):
    a = [df["confidence"][row] for df in dfs]
    labels_position.append(np.argmax(a))
    
label=[]
for j, i in enumerate(labels_position):
    label.append(dfs[i]["label_pred"][j])
    
df = dfs[0][["rewire_id"]]
df["label_pred"] = label

# df.to_csv("../ensemble.csv", index=False)
max_prob = f1_score(y_true, df.label_pred.values, average='macro')
print(max_prob)

0.8474320777279842


In [128]:
print(f"Soft Voting                    : {100 * soft:.2f}%")
print(f"Hard Voting                    : {100 * hard:.2f}%")
print(f"highest probability Voting     : {100 * max_prob:.2f}%")

Soft Voting                    : 84.92%
Hard Voting                    : 85.05%
highest probability Voting     : 84.74%
